In [ ]:
!pip install -q transformers
!pip install transformers
!pip install layoutlm
!pip install pdf2image
!pip install pytesseract
!pip install python-docx
!pip install opencv-python
!apt-get install poppler-utils
!apt-get install tesseract-ocr
!pip install jdc

ERROR: Could not find a version that satisfies the requirement layoutlm (from versions: none)
ERROR: No matching distribution found for layoutlm
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.5).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


Adding Debugger

In [ ]:
!pip install -Uqq ipdb
import ipdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.9 MB/s eta 0:00:00


In [ ]:
%pdb off

Automatic pdb calling has been turned OFF


In [ ]:
import os
import cv2
import numpy as np
import torch
from pdf2image import convert_from_path
from transformers import LayoutLMv3Processor, LayoutLMv3Model, LayoutLMv3ForSequenceClassification
from PIL import Image
import pytesseract
from typing import List, Dict, Tuple
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
file_path = "/content/drive/MyDrive/DeepLearning_FinalProject/MIT6390Notes_Chapter2.pdf"

In [ ]:
import fitz
import re
from dataclasses import dataclass
from typing import List, Dict, Tuple, Optional
import numpy as np
from transformers import LayoutLMv3Processor, LayoutLMv3Model
import torch
from PIL import Image
import re
import jdc

In [ ]:
@dataclass
class ContentElement:
    text: str
    bbox: List[float]
    type: str  # 'main_text', 'equation', 'margin_note', 'section_header', 'equation_reference'
    metadata: Dict = None
    confidence: float = 1.0
    page_num: int = 0

In [ ]:
class MIT690PDFExtractor:
    def __init__(self, layout_model_name="microsoft/layoutlmv3-base"):
        self.processor = LayoutLMv3Processor.from_pretrained(layout_model_name, apply_ocr=False)
        self.model = LayoutLMv3Model.from_pretrained(layout_model_name)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)

        # Patterns for MIT690 content
        self.patterns = {
            'equation_ref': r'\([\d\.]+\)',
            'section_number': r'^\d+\.\d+\s+[A-Z]',
            'equation': r'[=\+\-\*/∑∏∫]',
            'margin_note': r'(For example|Notice|You can think|Don\'t be)'
        }

In [ ]:
%%add_to MIT690PDFExtractor
def _classify_element(self, text: str, bbox: List[float],
                         img: Image) -> Tuple[str, Dict]:
        """Classify content element using rules and LayoutLMv3"""
        # Check patterns
        if re.search(self.patterns['margin_note'], text):
            return 'margin_notes', {'position': 'margin'}

        if re.search(self.patterns['section_number'], text):
            return 'section_headers', {'level': 1}

        if re.search(self.patterns['equation_ref'], text):
            return 'equation_references', {
                'equation_number': re.findall(r'\(([\d\.]+)\)', text)[0]
            }

        # Use LayoutLMv3 for more complex classification
        encoded = self.processor(
            img,
            text,
            boxes=[bbox],
            return_tensors="pt",
            truncation=True
        )

        encoded = {k: v.to(self.device) for k, v in encoded.items()}

        with torch.no_grad():
            outputs = self.model(**encoded)
            features = outputs.last_hidden_state[0].mean(dim=0)

            # Use features to detect equations
            if self._is_equation_from_features(features) or \
               re.search(self.patterns['equation'], text):
                return 'equations', {
                    'has_number': bool(re.search(r'\([\d\.]+\)', text))
                }

        return 'main_text', {}

In [ ]:
%%add_to MIT690PDFExtractor
def extract_content(self, pdf_path: str) -> Dict[str, List[ContentElement]]:
    """Extract structured content from academic PDF"""
    doc = fitz.open(pdf_path)
    content = {
        'main_text': [],
        'equations': [],
        'margin_notes': [],
        'section_headers': [],
        'equation_references': []
    }

    for page_num in range(len(doc)):
        page_elements = self.process_page(doc[page_num], page_num)
        for element in page_elements:
            content[element.type].append(element)

    return content

In [ ]:
%%add_to MIT690PDFExtractor
def process_page(self, page, page_num: int) -> List[ContentElement]:
    """Process single page and classify content elements"""
    elements = []

    # Convert page to image for LayoutLMv3
    pix = page.get_pixmap(matrix=fitz.Matrix(300/72, 300/72))
    img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)

    # Get text blocks with positioning
    blocks = page.get_text("dict")["blocks"]
    print("BLOCKS", blocks)
    for block in blocks:
        if "lines" not in block:
            continue

        text = []
        bounding_boxes = []

        # Iterate over lines and spans to extract words and their bounding boxes
        for line in block["lines"]:
            for span in line["spans"]:
                words = span["text"].split()  # Split span into individual words
                text.extend(words)  # Add each word to the list
                bbox = span["bbox"]
                bounding_boxes.extend([bbox] * len(words))  # Add the bounding box for each word


        print(text)
        print(bounding_boxes)
        #assert len(text) == len(bounding_boxes), f"Mismatch between words and bounding boxes!"

        element_type, metadata = self._classify_element(text, bounding_boxes, img)

        # Create ContentElement for each word
        for word, bbox in zip(text, bounding_boxes):
            element = ContentElement(
                text=word,
                bbox=bbox,
                type=element_type,
                metadata=metadata,
                page_num=page_num
            )
            elements.append(element)

        # Special handling for equations
        if element_type == 'main_text':
            equations = self._extract_inline_equations(" ".join(text), bounding_boxes[0], page_num)
            elements.extend(equations)

    return elements


In [ ]:
if __name__ == "__main__":
    extractor = MIT690PDFExtractor()
    content = extractor.extract_content(file_path)
    print(content)

BLOCKS [{'number': 0, 'type': 0, 'bbox': (386.0010070800781, 140.7171630859375, 468.00048828125, 169.86907958984375), 'lines': [{'spans': [{'size': 13.628800392150879, 'flags': 4, 'font': 'NimbusSanL-Regu', 'color': 0, 'ascender': 0.953000009059906, 'descender': -0.2849999964237213, 'text': 'CHAPTER', 'origin': (386.0010070800781, 163.15802001953125), 'bbox': (386.0010070800781, 150.16976928710938, 451.1194763183594, 167.04222106933594)}, {'size': 23.54759979248047, 'flags': 4, 'font': 'NimbusSanL-Regu', 'color': 0, 'ascender': 0.953000009059906, 'descender': -0.2849999964237213, 'text': ' 2', 'origin': (451.1194763183594, 163.15802001953125), 'bbox': (451.1194763183594, 140.7171630859375, 468.00048828125, 169.86907958984375)}], 'wmode': 0, 'dir': (1.0, 0.0), 'bbox': (386.0010070800781, 140.7171630859375, 468.00048828125, 169.86907958984375)}]}, {'number': 1, 'type': 0, 'bbox': (399.21600341796875, 232.66073608398438, 468.0005187988281, 249.53318786621094), 'lines': [{'spans': [{'size'

TypeError: expected string or bytes-like object